Target dataframe

filename | repository | workflow name | commit datetime | commit_id | action used | action version | vulnerable versions | fixing patch | Is Vulnerable | vulnerability disclosure

In [8]:
!pip install pyyaml
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 33.6 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 34.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [20]:
import yaml
import re

def extract_github_actions(yml_path):
    with open(yml_path, "r") as file:
        data = yaml.safe_load(file)

    actions = []

    def search_uses(obj):
        """Recursively search for 'uses' keys in the workflow YAML structure."""
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key == "uses" and isinstance(value, str):
                    # Match action and version, e.g. "actions/checkout@v3"
                    match = re.match(r"([^@]+)@(.+)", value)
                    if match:
                        actions.append({
                            "action": match.group(1),
                            "version": match.group(2)
                        })
                    else:
                        actions.append({"action": value, "version": "N/A"})
                else:
                    search_uses(value)
        elif isinstance(obj, list):
            for item in obj:
                search_uses(item)

    search_uses(data)
    return actions


if __name__ == "__main__":
    yml_path = "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml"  # e.g. ".github/workflows/ci.yml"
    actions = extract_github_actions(yml_path)

    print("🧩 GitHub Actions used in workflow:\n")
    print(len(actions), "actions found:\n")
    for act in actions:
        print(f"- {act['action']} (version: {act['version']})")

ParserError: while parsing a block mapping
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 10, column 5
expected <block end>, but found '-'
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 13, column 5

In [6]:
import re
from pathlib import Path

def parse_workflow_filename(full_path):
    path = Path(full_path)
    filename = path.name
    repository = str(path.parent)  # repository is the folder path

    # Regex to extract workflow name, datetime, and commit id
    pattern = r"^(?P<workflow>.+)__" \
              r"(?P<datetime>\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2}Z)__" \
              r"(?P<commit>[a-f0-9]{40})\.yml$"

    match = re.match(pattern, filename)
    if match:
        workflow_name = match.group("workflow")
        commit_datetime = match.group("datetime").replace("-", ":", 2).replace("-", "", 2)  # adjust hour-min-sec
        commit_datetime = commit_datetime.replace("-", ":", 2).replace("-", ":", 1)  # correct formatting to T13:53:55Z
        commit_id = match.group("commit")
        return {
            "filename": filename,
            "repository": repository,
            "workflow_name": workflow_name,
            "commit_datetime": commit_datetime,
            "commit_id": commit_id
        }
    else:
        return None

# Example usage
full_path = "yuri-moraes/sistema_erp/neon-preview-branches-for-pull-requests-gentle-field-50680377__2024-10-07T13-53-55Z__de100a5bff3f15e623b947464d458e0d641d774a.yml"

parsed = parse_workflow_filename(full_path)
print(parsed)

{'filename': 'neon-preview-branches-for-pull-requests-gentle-field-50680377__2024-10-07T13-53-55Z__de100a5bff3f15e623b947464d458e0d641d774a.yml', 'repository': 'yuri-moraes/sistema_erp', 'workflow_name': 'neon-preview-branches-for-pull-requests-gentle-field-50680377', 'commit_datetime': '2024:10:07T135355Z', 'commit_id': 'de100a5bff3f15e623b947464d458e0d641d774a'}


In [23]:
import os
import pandas as pd

# Assuming these two functions are already defined:
# 1. extract_github_actions(yml_path) -> returns list of {'action': ..., 'version': ...}
# 2. parse_workflow_filename(full_path) -> returns dict with filename, repository, workflow_name, commit_datetime, commit_id

def collect_workflow_data(root_folder):
    data = []
    num_correct_files = 0
    num_error_files = 0

    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.endswith(".yml") or file.endswith(".yaml"):
                full_path = os.path.join(dirpath, file)
                
                # Parse workflow filename info
                workflow_info = parse_workflow_filename(full_path)
                if workflow_info is None:
                    continue  # skip files that don't match the expected pattern

                # Extract GitHub actions from the YML
                # Try extracting GitHub actions safely
                try:
                    actions = extract_github_actions(full_path)
                except Exception as e:
                    num_error_files += 1
                    print(f"⚠️ Error processing file {full_path}: {e}")
                    data.append({
                        "filename": workflow_info["filename"],
                        "repository": workflow_info["repository"],
                        "workflow_name": workflow_info["workflow_name"],
                        "commit_datetime": workflow_info["commit_datetime"],
                        "commit_id": workflow_info["commit_id"],
                        "action_used": "ERROR",
                        "action_version": "ERROR",
                        "vulnerable_versions": None,
                        "fixing_patch": None,
                        "is_vulnerable": None
                    })
                    continue  # skip to the next file

                num_correct_files += 1

                # Combine each action with workflow info
                for act in actions:
                    data.append({
                        "filename": workflow_info["filename"],
                        "repository": workflow_info["repository"],
                        "workflow_name": workflow_info["workflow_name"],
                        "commit_datetime": workflow_info["commit_datetime"],
                        "commit_id": workflow_info["commit_id"],
                        "action_used": act.get("action"),
                        "action_version": act.get("version"),
                        "vulnerable_versions": None,
                        "fixing_patch": None,
                        "is_vulnerable": None
                    })

    print(f"✅ Processed {num_correct_files} files successfully.")
    print(f"⚠️ Encountered errors in {num_error_files} files.")

    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
root_folder = "dataset/workflows_from_dependents"
df = collect_workflow_data(root_folder)

# Preview
print(df.head())


⚠️ Error processing file dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml: while parsing a block mapping
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 10, column 5
expected <block end>, but found '-'
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-49-15Z__921f9f6c9c1b696031dbe8327f5bd9f79c0f9c97.yml", line 13, column 5
⚠️ Error processing file dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-50-54Z__567d7d276bf6755226a98649e3cb8f34b3953a83.yml: while parsing a block mapping
  in "dataset/workflows_from_dependents/Iamshankhadeep/novu/dev-deploy-web-component__2022-12-12T14-50-54Z__567d7d276bf6755226a98649e3cb8f34b3953a83.yml", line 10, column 5
expected <block end>, but found '-'
  i

In [24]:
df.head()

,filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
0,build.release__2021-02-06T21-24-22Z__be6e2564d...,dataset/workflows_from_dependents/MTaghaddomi/...,build.release,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/setup-buildx-action,v1,None,None,None
1,build.release__2021-02-06T21-24-22Z__be6e2564d...,dataset/workflows_from_dependents/MTaghaddomi/...,build.release,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/login-action,v1,None,None,None
2,build.release__2021-02-06T21-24-22Z__be6e2564d...,dataset/workflows_from_dependents/MTaghaddomi/...,build.release,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/build-push-action,v2,None,None,None
3,build.push__2021-02-06T21-24-22Z__be6e2564d6b1...,dataset/workflows_from_dependents/MTaghaddomi/...,build.push,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/setup-buildx-action,v1,None,None,None
4,build.push__2021-02-06T21-24-22Z__be6e2564d6b1...,dataset/workflows_from_dependents/MTaghaddomi/...,build.push,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/login-action,v1,None,None,None


In [25]:
df.to_csv("workflow_github_actions_analysis.csv", index=False)

In [26]:
df.head()

,filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
0,build.release__2021-02-06T21-24-22Z__be6e2564d...,dataset/workflows_from_dependents/MTaghaddomi/...,build.release,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/setup-buildx-action,v1,None,None,None
1,build.release__2021-02-06T21-24-22Z__be6e2564d...,dataset/workflows_from_dependents/MTaghaddomi/...,build.release,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/login-action,v1,None,None,None
2,build.release__2021-02-06T21-24-22Z__be6e2564d...,dataset/workflows_from_dependents/MTaghaddomi/...,build.release,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/build-push-action,v2,None,None,None
3,build.push__2021-02-06T21-24-22Z__be6e2564d6b1...,dataset/workflows_from_dependents/MTaghaddomi/...,build.push,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/setup-buildx-action,v1,None,None,None
4,build.push__2021-02-06T21-24-22Z__be6e2564d6b1...,dataset/workflows_from_dependents/MTaghaddomi/...,build.push,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,docker/login-action,v1,None,None,None


In [27]:
df.action_used.nunique()

1706

In [28]:
df_tjactions = df[df['action_used'].str.startswith('tj-actions/')]
df_tjactions.head()

,filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
6,build.push__2021-02-06T21-24-22Z__be6e2564d6b1...,dataset/workflows_from_dependents/MTaghaddomi/...,build.push,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,tj-actions/branch-names,v2,None,None,None
11,neon-preview-branches-for-pull-requests-fancy-...,dataset/workflows_from_dependents/miguelmasero...,neon-preview-branches-for-pull-requests-fancy-...,2025:01:14T010822Z,a84b1f16b0701920ef521dfd28435fdbe1ae9c29,tj-actions/branch-names,v8,None,None,None
17,pr-build__2024-08-20T09-32-13Z__d52e1d442b6cde...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:08:20T093213Z,d52e1d442b6cde25a557bdb5b8a6cc9fed59b6cc,tj-actions/branch-names,v8,None,None,None
36,pr-build__2024-05-21T09-40-37Z__34cbab63725550...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:05:21T094037Z,34cbab63725550479e2529453fb1bc619e75824d,tj-actions/branch-names,v8,None,None,None
55,pr-build__2024-02-27T15-54-27Z__089a236c9e2a72...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:02:27T155427Z,089a236c9e2a72529acad270dcfd20be9fac54fe,tj-actions/branch-names,v8,None,None,None


In [30]:
df_tjactions.action_used.unique()

array(['tj-actions/branch-names', 'tj-actions/changed-files',
       'tj-actions/sync-release-version',
       'tj-actions/github-changelog-generator',
       'tj-actions/eslint-changed-files',
       'tj-actions/verify-changed-files', 'tj-actions/auto-doc',
       'tj-actions/remark', 'tj-actions/release-tagger',
       'tj-actions/git-cliff', 'tj-actions/action-auto-doc',
       'tj-actions/bumpversion', 'tj-actions/pg-restore',
       'tj-actions/cargo-bump', 'tj-actions/coverage-badge-go',
       'tj-actions/pg-dump', 'tj-actions/branch-name',
       'tj-actions/semver-diff', 'tj-actions/branch_names'], dtype=object)

In [31]:
df_tjactions['action_used'].value_counts(dropna=False)

action_used
tj-actions/branch-names                  31018
tj-actions/verify-changed-files           7066
tj-actions/sync-release-version           5202
tj-actions/eslint-changed-files           3243
tj-actions/remark                         3186
tj-actions/github-changelog-generator     2972
tj-actions/release-tagger                 2206
tj-actions/changed-files                  1936
tj-actions/auto-doc                       1741
tj-actions/git-cliff                      1201
tj-actions/cargo-bump                      178
tj-actions/coverage-badge-go                79
tj-actions/pg-restore                       47
tj-actions/semver-diff                      23
tj-actions/pg-dump                           8
tj-actions/action-auto-doc                   4
tj-actions/bumpversion                       3
tj-actions/branch-name                       2
tj-actions/branch_names                      1
Name: count, dtype: int64

In [32]:
df_branch_names = df_tjactions[df_tjactions['action_used'] == 'tj-actions/branch-names']
df_branch_names.head(15)

,filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
6,build.push__2021-02-06T21-24-22Z__be6e2564d6b1...,dataset/workflows_from_dependents/MTaghaddomi/...,build.push,2021:02:06T212422Z,be6e2564d6b18bf9f15b842f2e4d9d894dbfdccc,tj-actions/branch-names,v2,None,None,None
11,neon-preview-branches-for-pull-requests-fancy-...,dataset/workflows_from_dependents/miguelmasero...,neon-preview-branches-for-pull-requests-fancy-...,2025:01:14T010822Z,a84b1f16b0701920ef521dfd28435fdbe1ae9c29,tj-actions/branch-names,v8,None,None,None
17,pr-build__2024-08-20T09-32-13Z__d52e1d442b6cde...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:08:20T093213Z,d52e1d442b6cde25a557bdb5b8a6cc9fed59b6cc,tj-actions/branch-names,v8,None,None,None
36,pr-build__2024-05-21T09-40-37Z__34cbab63725550...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:05:21T094037Z,34cbab63725550479e2529453fb1bc619e75824d,tj-actions/branch-names,v8,None,None,None
55,pr-build__2024-02-27T15-54-27Z__089a236c9e2a72...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:02:27T155427Z,089a236c9e2a72529acad270dcfd20be9fac54fe,tj-actions/branch-names,v8,None,None,None
74,publish__2024-02-07T15-57-34Z__45c42b65227e49a...,dataset/workflows_from_dependents/snyk-rogerio...,publish,2024:02:07T155734Z,45c42b65227e49a8cb3400cc1ba91ed98475a33c,tj-actions/branch-names,v8,None,None,None
85,pr-build__2024-08-29T09-06-01Z__9fe9ba73275402...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:08:29T090601Z,9fe9ba73275402ffb028ec07caf9b2777b099483,tj-actions/branch-names,v8,None,None,None
104,pr-build__2024-02-22T17-32-40Z__0020afaaed881f...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:02:22T173240Z,0020afaaed881fb3aaf1101bac23f51cd039c591,tj-actions/branch-names,v8,None,None,None
123,pr-build__2024-11-18T16-28-27Z__5b1765683a6100...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:11:18T162827Z,5b1765683a6100d55fc5da1638a9c76600b0e8d0,tj-actions/branch-names,v8,None,None,None
142,pr-build__2024-02-06T13-38-37Z__9ce23936a4ddd3...,dataset/workflows_from_dependents/snyk-rogerio...,pr-build,2024:02:06T133837Z,9ce23936a4ddd3b428123be3403fe079075d2bda,tj-actions/branch-names,v8,None,None,None


In [34]:
df_branch_names['action_used'].value_counts(dropna=False)

action_used
tj-actions/branch-names    31018
Name: count, dtype: int64

In [35]:
df_branch_names.to_csv("tj-actions-branch-names-usage.csv", index=False)

In [36]:
df_branch_names.rename(columns={'filename': 'yml_filename'}, inplace=True)

/var/folders/0y/1vh1g37j61j0kv4hcjwfgm2h0000gn/T/ipykernel_78139/1529449197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_branch_names.rename(columns={'filename': 'yml_filename'}, inplace=True)


In [63]:
version_names_branch_names_action = df_branch_names['action_version'].value_counts(dropna=False)

In [64]:
type(version_names_branch_names_action)

pandas.core.series.Series

In [67]:
version_names_branch_names_action

action_version
v8                                          6414
v6                                          6406
v7                                          6073
v5                                          3959
v5.2                                        2156
v4.8                                        1280
v5.1                                        1006
v4.9                                         491
v4.3                                         415
v7.0.7                                       385
6871f53176ad61624f978536bbf089c574dc19a2     341
v6.5                                         282
v4.5                                         278
v5.4                                         217
v8.0.1                                       197
v7.0.5                                       125
v6.1                                         123
v4                                           115
v6.4                                         108
v2.2                                          77
2e535

lets say latest is 8.2.4
then v8 will point to 8.2.4
v8.2 will point to 8.2.4

### This way we will be able to map the exact version of github-action being used using its commit SHA

In [70]:
# This code finds exactly which version of branch-names action is used for a given commit sha

sha = "eee8675bd61ec38bcfbfedd504d8473292ba649e"
variable = !cd branch-names-repo && git fetch --tags && git describe --tags --exact-match {sha}
variable

['v6.4']

### There is one repository using tj-actions/branch-names@main.

Lets have a deeper look into it...

In [72]:
df_branch_names[df_branch_names['action_version'] == 'main']

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
279482,build_apk__2024-11-10T11-58-59Z__87d89462d4088...,dataset/workflows_from_dependents/mrpoons-stud...,build_apk,2024:11:10T115859Z,87d89462d4088e6416032658b0c113233ea22fa1,tj-actions/branch-names,main,None,None,None
280207,build_apk__2024-11-10T11-50-34Z__72a42d9716fa1...,dataset/workflows_from_dependents/mrpoons-stud...,build_apk,2024:11:10T115034Z,72a42d9716fa15548c5bcddcaa2322e3b59ed066,tj-actions/branch-names,main,None,None,None
280311,build_apk__2024-11-10T11-43-15Z__f3bde7356d30b...,dataset/workflows_from_dependents/mrpoons-stud...,build_apk,2024:11:10T114315Z,f3bde7356d30b55e030e862099b8ea22c3491885,tj-actions/branch-names,main,None,None,None
280355,build_apk__2024-11-10T11-08-08Z__5a5cc003458b4...,dataset/workflows_from_dependents/mrpoons-stud...,build_apk,2024:11:10T110808Z,5a5cc003458b422d4d899a1bbcbe8196549457e7,tj-actions/branch-names,main,None,None,None


**Here's the addresses for these yml files:**

dataset/workflows_from_dependents/mrpoons-studio/kiwibrowser/build_apk__2024-11-10T11-08-08Z__5a5cc003458b422d4d899a1bbcbe8196549457e7.yml

dataset/workflows_from_dependents/mrpoons-studio/kiwibrowser/build_apk__2024-11-10T11-43-15Z__f3bde7356d30b55e030e862099b8ea22c3491885.yml

dataset/workflows_from_dependents/mrpoons-studio/kiwibrowser/build_apk__2024-11-10T11-50-34Z__72a42d9716fa15548c5bcddcaa2322e3b59ed066.yml

dataset/workflows_from_dependents/mrpoons-studio/kiwibrowser/build_apk__2024-11-10T11-58-59Z__87d89462d4088e6416032658b0c113233ea22fa1.yml

---

What tj-actions/branch-names@main means - 
“Run the latest code from the main branch of tj-actions/branch-names.”

**Its best that we remove @main version completely because number of such cases are too low to affect our study**

---

### First conclusion:

From this data we should be able to map each and every commit to its corresponding version now.

## Important Information:

This github-action was present in **National Vulnerability Database** twice in its lifetime.

1. https://github.com/advisories/GHSA-8v8w-v8xg-79rf - CVE-2023-49291
	1.	Vulnerability Detection Date: The exact date when the vulnerability was detected is not explicitly mentioned in the advisory.
	2.	Vulnerability Published Date: The advisory was published on **December 4, 2023**.
	3.	Fixing Patch Availability Date: The patch was made available on **December 7, 2023**, with the release of version v7.0.7.
    4. Affected Versions: All versions prior to **v7.0.7**
	5. Patched Version: **v7.0.7**

2. https://github.com/advisories/GHSA-8v8w-v8xg-79rf - CVE-2025-54416
	1. Vulnerability Detection Date: The exact date when the vulnerability was detected is not explicitly mentioned in the advisory.
	2. Advisory Published Date: **July 25, 2025**
	3. Fix Released Date: **July 28, 2025** (version v9.0.0)
    4. Affected Versions: All versions up to and including **v8.2.1**
	5. Patched Version: **v9.0.0**


1. Hence, every verson <v7.0.7 will be considered vulnerable from December 7, 2023 till today.
2. All versions >=v7.0.7 will be considered non-vulnerable from December 7, 2023 to July 28, 2025.
3. All versions <=v8.2.1 will be considered vulnerable from July 28, 2025 till today.
4. All versions >v8.2.1 will be considered non-vulnerable

In [89]:
df_branch_names[df_branch_names['action_version'] == '7.0.7']

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
103001,ci__2024-09-28T11-14-46Z__3ca012fbe8f5ba3771a1...,dataset/workflows_from_dependents/Tobi2K/TopTi...,ci,2024:09:28T111446Z,3ca012fbe8f5ba3771a1886bf378aa325013eef3,tj-actions/branch-names,7.0.7,None,None,None
1298541,on_push__2025-05-07T01-25-36Z__7edc5f3548faaf5...,dataset/workflows_from_dependents/coastal-scie...,on_push,2025:05:07T012536Z,7edc5f3548faaf5888dcc0cd02d557975ea49026,tj-actions/branch-names,7.0.7,None,None,None
1298559,on_push__2025-05-28T21-20-19Z__dc9a15886755d9c...,dataset/workflows_from_dependents/coastal-scie...,on_push,2025:05:28T212019Z,dc9a15886755d9cb6294f3015e3079b72164f665,tj-actions/branch-names,7.0.7,None,None,None


In [ ]:
df_branch_names[df_branch_names['action_version'] == '7.0.7']


These 2 repositories have wrong version of 7.0.7 (**not v7.0.7**)

Tobi2K/TopTips-Server

coastal-science/call-catalog-viewer

In [94]:
df_branch_names[
    (df_branch_names['repository'] == 'dataset/workflows_from_dependents/coastal-science/call-catalog-viewer') |
    (df_branch_names['repository'] == 'dataset/workflows_from_dependents/Tobi2K/TopTips-Server')
].head()

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
102928,build__2022-06-12T20-39-15Z__08b376af2158af72f...,dataset/workflows_from_dependents/Tobi2K/TopTi...,build,2022:06:12T203915Z,08b376af2158af72ffca37ea36d2811d991dce4e,tj-actions/branch-names,v5.1,None,None,None
102932,ci__2022-06-12T18-58-17Z__accd57e137371d0be9f3...,dataset/workflows_from_dependents/Tobi2K/TopTi...,ci,2022:06:12T185817Z,accd57e137371d0be9f3a7ac7c3f61203f605df7,tj-actions/branch-names,v5.1,None,None,None
102937,ci__2022-04-10T07-47-41Z__61315870d41a2050211b...,dataset/workflows_from_dependents/Tobi2K/TopTi...,ci,2022:04:10T074741Z,61315870d41a2050211b8534895b132849242f37,tj-actions/branch-names,v5.1,None,None,None
102942,ci__2022-06-12T20-39-15Z__08b376af2158af72ffca...,dataset/workflows_from_dependents/Tobi2K/TopTi...,ci,2022:06:12T203915Z,08b376af2158af72ffca37ea36d2811d991dce4e,tj-actions/branch-names,v5.1,None,None,None
102947,ci__2022-04-10T10-24-59Z__3a7662c7b961911a7b6c...,dataset/workflows_from_dependents/Tobi2K/TopTi...,ci,2022:04:10T102459Z,3a7662c7b961911a7b6c57014520e9ccbd189584,tj-actions/branch-names,v5.1,None,None,None


In [95]:
df_branch_names = df_branch_names.sort_values(
    by=['repository', 'workflow_name', 'commit_datetime'],
    ascending=[True, True, True]  # change any to False if you want descending order
)

In [100]:
df_branch_names = df_branch_names.reset_index(drop=True)
df_branch_names.head()

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
0,neon-preview-branches-for-pull-requests-shrill...,dataset/workflows_from_dependents/007ekho/pos_...,neon-preview-branches-for-pull-requests-shrill...,2025:01:08T032639Z,2e0d6d422849095b63e8dd76347e32e6e78e0fec,tj-actions/branch-names,v8,None,None,None
1,pr__2025-01-22T10-18-57Z__768f732f17a18d2173ca...,dataset/workflows_from_dependents/0OZ/mono-sta...,pr,2025:01:22T101857Z,768f732f17a18d2173ca32b01efc858eebf44045,tj-actions/branch-names,v8,None,None,None
2,pr-cleanup__2025-01-22T10-18-57Z__768f732f17a1...,dataset/workflows_from_dependents/0OZ/mono-sta...,pr-cleanup,2025:01:22T101857Z,768f732f17a18d2173ca32b01efc858eebf44045,tj-actions/branch-names,v8,None,None,None
3,Bump Version__2023-02-10T22-47-16Z__643b50eca8...,dataset/workflows_from_dependents/1337-server/...,Bump Version,2023:02:10T224716Z,643b50eca803df36454f652023f033442d6f75ca,tj-actions/branch-names,v6,None,None,None
4,Bump Version__2023-02-16T04-51-47Z__a73466bd49...,dataset/workflows_from_dependents/1337-server/...,Bump Version,2023:02:16T045147Z,a73466bd49e8aac9cfefff25731df588b8fa1398,tj-actions/branch-names,v6,None,None,None


#### Replacing wrong version (7.0.7 - not v7.0.7) entries with previous commit's version

In [102]:
for idx, row in df_branch_names.iterrows():
    if row['action_version'] == '7.0.7':
        prev_idx = idx - 1
        if prev_idx >= 0 and df_branch_names.at[prev_idx, 'repository'] == row['repository'] and df_branch_names.at[prev_idx, 'workflow_name'] == row['workflow_name']:
            print(f"Replacing index {idx} version '7.0.7' with previous index {prev_idx} version '{df_branch_names.at[prev_idx, 'action_version']}'")
            df_branch_names.at[idx, 'action_version'] = df_branch_names.at[prev_idx, 'action_version']

Replacing index 7195 version '7.0.7' with previous index 7194 version 'v5.1'
Replacing index 12049 version '7.0.7' with previous index 12048 version 'v6'
Replacing index 12051 version '7.0.7' with previous index 12050 version 'v6'


The reason for updating it to previous index's version was because the build for 7.0.7 would have failed and current production instance would be carrying previous correct version.

One more question to ask is is the version_id they are using are actually correct?

In [103]:
df_branch_names[df_branch_names['action_version'] == '7.0.7']

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable


#### Removing entries with action_version = main

In [75]:
df_branch_names = df_branch_names[df_branch_names['action_version'] != 'main']

In [104]:
version_pattern_inclusive = r'^v\d+(\.\d+){0,2}$'

for idx, row in df_branch_names.iterrows():
    action_version = row['action_version']
    if not re.match(version_pattern_inclusive, action_version):
        print(f"Non-standard version found: {action_version} in row {idx}")

# sha = "eee8675bd61ec38bcfbfedd504d8473292ba649e"
# variable = !cd branch-names-repo && git fetch --tags && git describe --tags --exact-match {sha}
# variable

Non-standard version found: 6c999acf206f5561e19f46301bb310e9e70d8815 in row 234
Non-standard version found: 6871f53176ad61624f978536bbf089c574dc19a2 in row 1088
Non-standard version found: 6871f53176ad61624f978536bbf089c574dc19a2 in row 1746
Non-standard version found: 6c999acf206f5561e19f46301bb310e9e70d8815 in row 3120
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3144
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3145
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3146
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3147
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3148
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3149
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3150
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3151
Non-standard version found: e

In [106]:
df_branch_names_backup = df_branch_names.copy()
df_branch_names_backup.to_csv("tj-actions-branch-names-usage-cleaned.csv", index=False)

In [108]:
version_pattern_inclusive = r'^v\d+(\.\d+){0,2}$'

for idx, row in df_branch_names.iterrows():
    action_version = row['action_version']
    if not re.match(version_pattern_inclusive, action_version):
        variable = !cd branch-names-repo && git fetch --tags && git describe --tags --exact-match {action_version}
        print(f"Non-standard version found: {action_version} in row {idx}, resolved to tag: {variable[0]}")
        df_branch_names.at[idx, 'action_version'] = variable[0]

# sha = "eee8675bd61ec38bcfbfedd504d8473292ba649e"
# variable = !cd branch-names-repo && git fetch --tags && git describe --tags --exact-match {sha}
# variable

Non-standard version found: 6c999acf206f5561e19f46301bb310e9e70d8815 in row 234, resolved to tag: v7
Non-standard version found: 6871f53176ad61624f978536bbf089c574dc19a2 in row 1088, resolved to tag: v8.0.1
Non-standard version found: 6871f53176ad61624f978536bbf089c574dc19a2 in row 1746, resolved to tag: v8.0.1
Non-standard version found: 6c999acf206f5561e19f46301bb310e9e70d8815 in row 3120, resolved to tag: v7
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3144, resolved to tag: v6.4
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3145, resolved to tag: v6.4
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3146, resolved to tag: v6.4
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3147, resolved to tag: v6.4
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba649e in row 3148, resolved to tag: v6.4
Non-standard version found: eee8675bd61ec38bcfbfedd504d8473292ba6

#### Checking all rows for which action version does not match r'^v\d+(\.\d+){0,2}$'

In [112]:
df_branch_names[~df_branch_names['action_version'].str.match(r'^v\d+(\.\d+){0,2}$', na=False)]

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable


### Now that we have all the versioning corrected, lets start the analysis

1. Every verson <v7.0.7 will be considered vulnerable from December 7, 2023 till today.
2. All versions >=v7.0.7 will be considered non-vulnerable from December 7, 2023 to July 28, 2025.
3. All versions <=v8.2.1 will be considered vulnerable from July 28, 2025 till today.
4. All versions >v8.2.1 will be considered non-vulnerable

In [113]:
# Lets first save this cleaned dataframe as csv
df_branch_names.to_csv("tj-actions-branch-names-usage-cleaned.csv", index=False)

In [114]:
df_branch_names.columns

Index(['yml_filename', 'repository', 'workflow_name', 'commit_datetime',
       'commit_id', 'action_used', 'action_version', 'vulnerable_versions',
       'fixing_patch', 'is_vulnerable'],
      dtype='object')

column vulnerable_versions will take values:
- none if commit date was before Dec 7, 2023
- <v7.0.7 if commit date >=Dec 7, 2023 but <July 27, 2025
- <=v8.2.1 if commit date >=July 28, 2025


In [115]:
df_branch_names.head()

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable
0,neon-preview-branches-for-pull-requests-shrill...,dataset/workflows_from_dependents/007ekho/pos_...,neon-preview-branches-for-pull-requests-shrill...,2025:01:08T032639Z,2e0d6d422849095b63e8dd76347e32e6e78e0fec,tj-actions/branch-names,v8,None,None,None
1,pr__2025-01-22T10-18-57Z__768f732f17a18d2173ca...,dataset/workflows_from_dependents/0OZ/mono-sta...,pr,2025:01:22T101857Z,768f732f17a18d2173ca32b01efc858eebf44045,tj-actions/branch-names,v8,None,None,None
2,pr-cleanup__2025-01-22T10-18-57Z__768f732f17a1...,dataset/workflows_from_dependents/0OZ/mono-sta...,pr-cleanup,2025:01:22T101857Z,768f732f17a18d2173ca32b01efc858eebf44045,tj-actions/branch-names,v8,None,None,None
3,Bump Version__2023-02-10T22-47-16Z__643b50eca8...,dataset/workflows_from_dependents/1337-server/...,Bump Version,2023:02:10T224716Z,643b50eca803df36454f652023f033442d6f75ca,tj-actions/branch-names,v6,None,None,None
4,Bump Version__2023-02-16T04-51-47Z__a73466bd49...,dataset/workflows_from_dependents/1337-server/...,Bump Version,2023:02:16T045147Z,a73466bd49e8aac9cfefff25731df588b8fa1398,tj-actions/branch-names,v6,None,None,None


In [116]:
# Extract date part and convert to datetime
df_branch_names['commit_date'] = pd.to_datetime(df['commit_datetime'].str[:10].str.replace(':', '-'))

df_branch_names['commit_date'].head()

0   2021-02-06
1   2021-02-06
2   2021-02-06
3   2021-02-06
4   2021-02-06
Name: commit_date, dtype: datetime64[ns]

In [120]:
import pandas as pd

# Ensure commit_date is in datetime format
df_branch_names['commit_date'] = pd.to_datetime(df_branch_names['commit_date'])

# Define the date thresholds
date1 = pd.Timestamp('2023-12-07')
date2 = pd.Timestamp('2025-07-27')

# Apply the logic
df_branch_names['vulnerable_versions'] = pd.cut(
    df_branch_names['commit_date'],
    bins=[pd.Timestamp.min, date1, date2, pd.Timestamp.max],
    labels=['none', '<v7.0.7', '<=v8.2.1'],
    right=False  # ensures >= logic for boundaries
)

df_branch_names[['action_version','commit_date', 'vulnerable_versions']].sample(10)


,action_version,commit_date,vulnerable_versions
558,v4.8,2023-01-20,none
10031,v7,2020-11-26,none
7982,v6,2021-06-01,none
1569,v6,2022-12-26,none
14722,v6,2022-09-20,none
2200,v8,2022-11-20,none
29813,v4.8,2020-10-30,none
25605,v8.0.1,2021-02-06,none
24811,v7,2025-01-16,<v7.0.7
30619,v7.0.7,2021-06-09,none


In [144]:
df_branch_names[['action_version','commit_date', 'vulnerable_versions']].sample(10)


,action_version,commit_date,vulnerable_versions
7414,v8,2021-09-20,none
9727,v8,2020-11-21,none
24222,v7,2022-11-12,none
22355,v8,2022-09-20,none
1693,v7,2023-01-12,none
12805,v5.1,2022-10-14,none
8211,v7.0.5,2021-11-17,none
4612,v8,2022-07-24,none
21273,v5.1,2021-05-12,none
23950,v7,2022-07-19,none


In [162]:
df_with_open_vulenrabilities = df_branch_names[df_branch_names['vulnerable_versions'] != 'none']


In [ ]:
df_with_open_vulenrabilities.nunique()

yml_filename           2978
repository              419
workflow_name           216
commit_datetime        2838
commit_id              2931
action_used               1
action_version           29
vulnerable_versions       1
fixing_patch              0
is_vulnerable             0
commit_date             210
dtype: int64

In [165]:
df_with_open_vulenrabilities[df_with_open_vulenrabilities['vulnerable_versions'] == '<=v8.2.1'].head()


,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable,commit_date


#### From above we see that there is no commit date in dataset that might vulnerable to latest issue presented on 28 July 2025.

Lets also verify that there is no commit beyond 28 july 2025

In [166]:
# Largest date in commit_date column
latest_commit = df_with_open_vulenrabilities['commit_date'].max()

print(latest_commit)

2025-05-19 00:00:00


In [170]:
df_with_open_vulenrabilities['fixing_patch'] = 'v7.0.7'

/var/folders/0y/1vh1g37j61j0kv4hcjwfgm2h0000gn/T/ipykernel_78139/3638516486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_open_vulenrabilities['fixing_patch'] = 'v7.0.7'


In [174]:
def compare_versions(v1, v2):
    """
    Compare two version strings like 'v8.0.0' and 'v7.0.7'.
    Returns:
        1 if v1 > v2
        -1 if v1 < v2
        0 if v1 == v2
    """
    # Remove leading 'v' if present and split into integers
    parts1 = [int(x) for x in v1.lstrip('v').split('.')]
    parts2 = [int(x) for x in v2.lstrip('v').split('.')]

    # Pad shorter list with zeros to handle uneven lengths like v8.0 vs v8.0.0
    max_len = max(len(parts1), len(parts2))
    parts1 += [0] * (max_len - len(parts1))
    parts2 += [0] * (max_len - len(parts2))

    # Compare part by part
    if parts1 > parts2:
        return 1
    elif parts1 < parts2:
        return -1
    else:
        return 0
    
print(compare_versions('v8.0.0', 'v7.0.7'))  # Should return 1
print(compare_versions('v7.0.7', 'v8.0.0'))  # Should return -1
print(compare_versions('v8.0.0', 'v8.0.0'))  # Should return 0

print(compare_versions('v8.0.0', 'v8'))  # Should return 0
print(compare_versions('v7.0.7', 'v8'))  # Should return -1
print(compare_versions('v8.0.1', 'v8'))  # Should return 1

print(compare_versions('v8.0.9', 'v8.1'))  # Should return -1
print(compare_versions('v8.1', 'v8.0.9'))  # Should return 1
print(compare_versions('v8.1.0', 'v8.1'))  # Should return 0
print(compare_versions('v8.1.1', 'v8.1'))  # Should return 1

print(compare_versions('v8', 'v8.1'))  # Should return -1
print(compare_versions('v8.1.1', 'v8.1'))  # Should return 1

1
-1
0
0
-1
1
-1
1
0
1
-1
1


In [ ]:
import pandas as pd

# Apply comparison logic
df_with_open_vulenrabilities['is_vulnerable'] = df_with_open_vulenrabilities.apply(
    lambda row: 0 if compare_versions(row['action_version'], row['fixing_patch']) >= 0 else 1,
    axis=1
)



/var/folders/0y/1vh1g37j61j0kv4hcjwfgm2h0000gn/T/ipykernel_78139/3622157147.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_open_vulenrabilities['is_vulnerable'] = df_with_open_vulenrabilities.apply(


,commit_date,action_version,vulnerable_versions,fixing_patch,is_vulnerable
25848,2024-08-29,v6,<v7.0.7,v7.0.7,1
10488,2024-03-28,v6,<v7.0.7,v7.0.7,1
23604,2025-02-05,v8,<v7.0.7,v7.0.7,0
25786,2024-07-09,v6,<v7.0.7,v7.0.7,1
19769,2024-01-31,v8,<v7.0.7,v7.0.7,0
119,2024-02-22,v7.0.5,<v7.0.7,v7.0.7,1
17178,2024-07-22,v5.2,<v7.0.7,v7.0.7,1
10539,2024-04-26,v7,<v7.0.7,v7.0.7,1
27123,2024-07-01,v7,<v7.0.7,v7.0.7,1
13504,2024-02-20,v5.1,<v7.0.7,v7.0.7,1


In [190]:
df_with_open_vulenrabilities[['commit_date', 'action_version', 'vulnerable_versions', 'fixing_patch', 'is_vulnerable']].sample(10)

,commit_date,action_version,vulnerable_versions,fixing_patch,is_vulnerable
22541,2024-05-12,v7,<v7.0.7,v7.0.7,1
13412,2024-02-20,v4.5,<v7.0.7,v7.0.7,1
18289,2024-02-01,v7,<v7.0.7,v7.0.7,1
19212,2024-06-13,v8,<v7.0.7,v7.0.7,0
24469,2024-05-30,v7,<v7.0.7,v7.0.7,1
10406,2024-04-12,v5,<v7.0.7,v7.0.7,1
22705,2024-02-03,v5.2,<v7.0.7,v7.0.7,1
21251,2024-05-08,v8,<v7.0.7,v7.0.7,0
23041,2024-03-08,v7,<v7.0.7,v7.0.7,1
23568,2024-11-18,v6,<v7.0.7,v7.0.7,1


In [192]:
df_with_open_vulenrabilities[df_with_open_vulenrabilities['action_version'] >= 'v7.0.7'].nunique()

yml_filename           899
repository             165
workflow_name          101
commit_datetime        881
commit_id              887
action_used              1
action_version           6
vulnerable_versions      1
fixing_patch             1
is_vulnerable            1
commit_date            130
dtype: int64

In [193]:
df_with_open_vulenrabilities[df_with_open_vulenrabilities['action_version'] < 'v7.0.7'].nunique()

yml_filename           2080
repository              303
workflow_name           141
commit_datetime        1959
commit_id              2046
action_used               1
action_version           23
vulnerable_versions       1
fixing_patch              1
is_vulnerable             1
commit_date             199
dtype: int64

In [196]:
df_with_open_vulenrabilities.to_csv("tj-actions-branch-names-usage-with-vuln-analysis.csv", index=False)

In [3]:
import pandas as pd

df_with_open_vulenrabilities = pd.read_csv("tj-actions-branch-names-usage-with-vuln-analysis.csv")

In [5]:
len(df_with_open_vulenrabilities)

5827

In [8]:
df_with_open_vulenrabilities.head()

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable,commit_date
0,deploy-docker__2023-02-11T01-05-26Z__dd829610d...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023:02:11T010526Z,dd829610d30cd16ccfb2d15b88a44e9188c5b36e,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
1,deploy-docker__2023-02-11T09-54-02Z__0065a4094...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023:02:11T095402Z,0065a4094ed1b19a95a26baa4411c78558d3e3f0,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
2,deploy-docker__2023-02-11T09-56-37Z__aad26d664...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023:02:11T095637Z,aad26d664ece4be9fb0cea77dd2086bc9c2e53d7,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
3,deploy-docker__2023-02-11T10-03-38Z__4a6182c5c...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023:02:11T100338Z,4a6182c5c7d00a12b1d1ddc52784da45bd64eadb,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-13
4,deploy-docker__2023-02-11T10-07-29Z__61f5772d3...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023:02:11T100729Z,61f5772d37cb0af5ea82be024d3eff1b5ca88bcd,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-13


In [7]:
df_with_open_vulenrabilities['repository'].nunique()

419

In [10]:
df_with_open_vulenrabilities[['repository', 'workflow_name']].nunique()

repository       419
workflow_name    216
dtype: int64

In [11]:
df_with_open_vulenrabilities['commit_datetime'] = pd.to_datetime(
    df_with_open_vulenrabilities['commit_datetime'],
    format='%Y:%m:%dT%H%M%SZ',   # Matches pattern '2023:02:11T010526Z'
    errors='coerce'              # Converts invalid values to NaT instead of raising error
)

In [12]:
df_with_open_vulenrabilities.head()

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable,commit_date
0,deploy-docker__2023-02-11T01-05-26Z__dd829610d...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 01:05:26,dd829610d30cd16ccfb2d15b88a44e9188c5b36e,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
1,deploy-docker__2023-02-11T09-54-02Z__0065a4094...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 09:54:02,0065a4094ed1b19a95a26baa4411c78558d3e3f0,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
2,deploy-docker__2023-02-11T09-56-37Z__aad26d664...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 09:56:37,aad26d664ece4be9fb0cea77dd2086bc9c2e53d7,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
3,deploy-docker__2023-02-11T10-03-38Z__4a6182c5c...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 10:03:38,4a6182c5c7d00a12b1d1ddc52784da45bd64eadb,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-13
4,deploy-docker__2023-02-11T10-07-29Z__61f5772d3...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 10:07:29,61f5772d37cb0af5ea82be024d3eff1b5ca88bcd,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-13


In [14]:
df_with_open_vulenrabilities = df_with_open_vulenrabilities.sort_values(by=['repository', 'workflow_name', 'commit_datetime'], ascending=True)

In [20]:
df_with_open_vulenrabilities.iloc[0]['repository']

'dataset/workflows_from_dependents/1337-server/arm-dependencies'

In [21]:
df_with_open_vulenrabilities[df_with_open_vulenrabilities['repository'] == 'dataset/workflows_from_dependents/1337-server/arm-dependencies']

,yml_filename,repository,workflow_name,commit_datetime,commit_id,action_used,action_version,vulnerable_versions,fixing_patch,is_vulnerable,commit_date
0,deploy-docker__2023-02-11T01-05-26Z__dd829610d...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 01:05:26,dd829610d30cd16ccfb2d15b88a44e9188c5b36e,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
1,deploy-docker__2023-02-11T09-54-02Z__0065a4094...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 09:54:02,0065a4094ed1b19a95a26baa4411c78558d3e3f0,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
2,deploy-docker__2023-02-11T09-56-37Z__aad26d664...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 09:56:37,aad26d664ece4be9fb0cea77dd2086bc9c2e53d7,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-14
3,deploy-docker__2023-02-11T10-03-38Z__4a6182c5c...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 10:03:38,4a6182c5c7d00a12b1d1ddc52784da45bd64eadb,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-13
4,deploy-docker__2023-02-11T10-07-29Z__61f5772d3...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 10:07:29,61f5772d37cb0af5ea82be024d3eff1b5ca88bcd,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2025-01-13
5,deploy-docker__2023-02-11T19-53-27Z__c2e21bc4d...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 19:53:27,c2e21bc4def52ab10a0d96257c3c594f6e854505,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2024-08-20
6,deploy-docker__2023-02-11T19-55-11Z__35a159dcc...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 19:55:11,35a159dcc6e814f3057640ef39efd7a443a87a54,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2024-08-20
7,deploy-docker__2023-02-11T20-01-27Z__295851f98...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 20:01:27,295851f9805b35fc5ccd23293fbc58abdcb95741,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2024-08-20
8,deploy-docker__2023-02-11T20-04-23Z__c29280df3...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 20:04:23,c29280df3fd6b622cefe034792217cd0b595c770,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2024-08-20
9,deploy-docker__2023-02-11T20-09-46Z__151c43a39...,dataset/workflows_from_dependents/1337-server/...,deploy-docker,2023-02-11 20:09:46,151c43a398e63f9661c6485f865f7cae0260021e,tj-actions/branch-names,v6,<v7.0.7,v7.0.7,1,2024-08-20


In [23]:
df_with_open_vulenrabilities['commit_date'] = df_with_open_vulenrabilities['commit_datetime'].dt.date

In [24]:
df_with_open_vulenrabilities.to_csv("tj-actions-branch-names-usage-with-vuln-analysis.csv", index=False)

In [25]:
len(df_with_open_vulenrabilities[['repository', 'workflow_name']].drop_duplicates())

484

In [29]:
from datetime import datetime

def compute_vuln_duration(group):
    group = group.sort_values('commit_date')
    
    # Find first date where it became vulnerable (is_vulnerable == 1)
    first_vuln = group.loc[group['is_vulnerable'] == 1, 'commit_date'].min()
    
    if pd.isna(first_vuln):
        # Never vulnerable
        return pd.Series({'vuln_start': None, 'vuln_end': None, 'vuln_days': 0})
    
    # Find the first date *after* first_vuln where it became non-vulnerable (0)
    after_vuln = group[group['commit_date'] > first_vuln]
    first_fixed = after_vuln.loc[after_vuln['is_vulnerable'] == 0, 'commit_date'].min()
    
    # If never fixed, assume stays vulnerable until 2025-05-19
    if pd.isna(first_fixed):
        vuln_end = pd.Timestamp('2025-05-19')
    else:
        vuln_end = first_fixed
    
    # Calculate duration in days
    duration = (vuln_end - first_vuln).days
    
    return pd.Series({
        'vuln_start': first_vuln,
        'vuln_end': vuln_end,
        'vuln_days': duration
    })

In [30]:



df_with_open_vulenrabilities['commit_date'] = pd.to_datetime(df_with_open_vulenrabilities['commit_date'])
# df = df.sort_values(['repository', 'workflow_name', 'commit_date'])

result = df_with_open_vulenrabilities.groupby(['repository','workflow_name']).apply(compute_vuln_duration).reset_index()
print(result)

                                            repository         workflow_name  \
0    dataset/workflows_from_dependents/1337-server/...         deploy-docker   
1    dataset/workflows_from_dependents/189569400/Op...  gh-branch-name-check   
2    dataset/workflows_from_dependents/1NF1N18Y/sma...         release_alpha   
3    dataset/workflows_from_dependents/2949454190/m...             snapshots   
4    dataset/workflows_from_dependents/362190147/mirai             snapshots   
..                                                 ...                   ...   
479  dataset/workflows_from_dependents/vk-gh-action...                  test   
480  dataset/workflows_from_dependents/workivate/ch...                  test   
481  dataset/workflows_from_dependents/yagcioe/ngx-...           publish_npm   
482  dataset/workflows_from_dependents/zendesk/chan...                  test   
483  dataset/workflows_from_dependents/zendesk/json...  sync-release-version   

    vuln_start   vuln_end  vuln_days  


/var/folders/0y/1vh1g37j61j0kv4hcjwfgm2h0000gn/T/ipykernel_3461/3961969459.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df_with_open_vulenrabilities.groupby(['repository','workflow_name']).apply(compute_vuln_duration).reset_index()


In [31]:
result.to_csv("vuln-durations.csv", index=False)

In [34]:
df_with_open_vulenrabilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5827 entries, 0 to 5826
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   yml_filename         5827 non-null   object        
 1   repository           5827 non-null   object        
 2   workflow_name        5827 non-null   object        
 3   commit_datetime      5827 non-null   datetime64[ns]
 4   commit_id            5827 non-null   object        
 5   action_used          5827 non-null   object        
 6   action_version       5827 non-null   object        
 7   vulnerable_versions  5827 non-null   object        
 8   fixing_patch         5827 non-null   object        
 9   is_vulnerable        5827 non-null   int64         
 10  commit_date          5827 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 500.9+ KB


In [35]:
df_with_open_vulenrabilities['is_vulnerable'] = df_with_open_vulenrabilities['is_vulnerable'].astype(bool)

In [36]:
df_with_open_vulenrabilities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5827 entries, 0 to 5826
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   yml_filename         5827 non-null   object        
 1   repository           5827 non-null   object        
 2   workflow_name        5827 non-null   object        
 3   commit_datetime      5827 non-null   datetime64[ns]
 4   commit_id            5827 non-null   object        
 5   action_used          5827 non-null   object        
 6   action_version       5827 non-null   object        
 7   vulnerable_versions  5827 non-null   object        
 8   fixing_patch         5827 non-null   object        
 9   is_vulnerable        5827 non-null   bool          
 10  commit_date          5827 non-null   datetime64[ns]
dtypes: bool(1), datetime64[ns](2), object(8)
memory usage: 461.1+ KB


In [37]:
# Ensure commit_datetime is sorted
df_sorted = df_with_open_vulenrabilities.sort_values(["repository", "workflow_name", "commit_datetime"])

# Pick the last commit for each (repository, workflow_name)
final_state = df_sorted.groupby(["repository", "workflow_name"]).tail(1)

# Now separate into two categories
vulnerable_at_end = final_state[final_state["is_vulnerable"] == True]
not_vulnerable_at_end = final_state[final_state["is_vulnerable"] == False]

In [46]:
vulnerable_combinations = vulnerable_at_end[["repository", "workflow_name"]].values.tolist()

not_vulnerable_combinations = not_vulnerable_at_end[["repository", "workflow_name"]].values.tolist()

In [41]:
print(f"Number of workflows still vulnerable at the end: {len(vulnerable_at_end)}")
print(f"Number of workflows not vulnerable at the end: {len(not_vulnerable_at_end)}")

Number of workflows still vulnerable at the end: 301
Number of workflows not vulnerable at the end: 183


In [48]:
# Convert combinations into a MultiIndex
vul_idx = pd.MultiIndex.from_tuples(vulnerable_combinations, names=["repository", "workflow_name"])
not_vul_idx = pd.MultiIndex.from_tuples(not_vulnerable_combinations, names=["repository", "workflow_name"])

# Set df index to the same columns
df_indexed = df_with_open_vulenrabilities.set_index(["repository", "workflow_name"])

# Extract rows matching vulnerable combinations
df_vulnerable = df_indexed.loc[df_indexed.index.isin(vul_idx)].reset_index()

# Extract rows matching non-vulnerable combinations
df_not_vulnerable = df_indexed.loc[df_indexed.index.isin(not_vul_idx)].reset_index()

In [50]:
print(len(df_vulnerable))
print(len(df_not_vulnerable))
print(len(df_vulnerable) + len(df_not_vulnerable))
print(len(df_with_open_vulenrabilities))

2977
2850
5827
5827


In [ ]:
df_vulnerable.to_csv("non_fixed_workflows.csv", index=False)
df_not_vulnerable.to_csv("fixed_workflows.csv", index=False)

In [67]:
def analyze_vulnerability_period(group):
    """
    Given a dataframe block for a single (repository, workflow_name),
    sorted by commit_datetime, this function returns:
    - date when vulnerability first appeared
    - date when vulnerability was permanently fixed
    - total number of days vulnerable
    - vulnerability intervals
    """
    group = group.sort_values("commit_datetime").reset_index(drop=True)

    intervals = []     # list of (start, end)
    vulnerable = False
    start_date = None

    for i in range(len(group)):
        state = group.loc[i, "is_vulnerable"]
        date = group.loc[i, "commit_date"]

        # vulnerable begins
        if state and not vulnerable:
            vulnerable = True
            start_date = date

        # vulnerable ends
        elif not state and vulnerable:
            vulnerable = False
            end_date = date
            intervals.append((start_date, end_date))
            start_date = None

    # if it ends still vulnerable → treat as open interval
    if vulnerable:
        intervals.append((start_date, None))

    # --- Compute summary ---
    if intervals:
        first_vulnerability_date = intervals[0][0]
    else:
        first_vulnerability_date = None

    # permanent fix = last interval's end date
    if intervals and intervals[-1][1] is not None:
        permanent_fix_date = intervals[-1][1]
    else:
        permanent_fix_date = None

    # total days vulnerable (sum all closed intervals)
    total_days = 0
    for start, end in intervals:
        if end is not None:
            total_days += (end - start).days

    return pd.Series({
        "first_vulnerability_date": first_vulnerability_date,
        "permanent_fix_date": permanent_fix_date,
        "total_days_vulnerable": total_days,
        "intervals": intervals
    })

In [68]:
repo = "dataset/workflows_from_dependents/3dwardch3ng/delete-from-s3-action"
workflow = "ci"

block = df_not_vulnerable[(df_not_vulnerable["repository"] == repo) & 
           (df_not_vulnerable["workflow_name"] == workflow)]

print(block)

                                           repository workflow_name  \
5   dataset/workflows_from_dependents/3dwardch3ng/...            ci   
6   dataset/workflows_from_dependents/3dwardch3ng/...            ci   
7   dataset/workflows_from_dependents/3dwardch3ng/...            ci   
8   dataset/workflows_from_dependents/3dwardch3ng/...            ci   
9   dataset/workflows_from_dependents/3dwardch3ng/...            ci   
10  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
11  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
12  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
13  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
14  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
15  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
16  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
17  dataset/workflows_from_dependents/3dwardch3ng/...            ci   
18  da

In [69]:

result = analyze_vulnerability_period(block)
print(result)

first_vulnerability_date                                  2023-09-02 00:00:00
permanent_fix_date                                        2023-12-19 00:00:00
total_days_vulnerable                                                      94
intervals                   [(2023-09-02 00:00:00, 2023-12-05 00:00:00), (...
dtype: object


In [70]:
summary_df = (
    df_not_vulnerable.groupby(["repository", "workflow_name"])
      .apply(analyze_vulnerability_period)
      .reset_index()
)

/var/folders/0y/1vh1g37j61j0kv4hcjwfgm2h0000gn/T/ipykernel_3461/3205024386.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(analyze_vulnerability_period)


In [71]:
summary_df.head()

,repository,workflow_name,first_vulnerability_date,permanent_fix_date,total_days_vulnerable,intervals
0,dataset/workflows_from_dependents/189569400/Op...,gh-branch-name-check,NaT,NaT,0,[]
1,dataset/workflows_from_dependents/3dwardch3ng/...,ci,2023-09-02,2023-12-19,94,"[(2023-09-02 00:00:00, 2023-12-05 00:00:00), (..."
2,dataset/workflows_from_dependents/3dwardch3ng/...,ci,2023-09-05,2023-12-05,91,"[(2023-09-05 00:00:00, 2023-12-05 00:00:00)]"
3,dataset/workflows_from_dependents/3dwardch3ng/...,github-action-test,2023-09-20,2023-12-05,76,"[(2023-09-20 00:00:00, 2023-12-05 00:00:00)]"
4,dataset/workflows_from_dependents/4x3l3r8/what...,neon-preview-branches-for-pull-requests-white-...,NaT,NaT,0,[]


In [ ]:
# Split the path and take last 2 parts
summary_df["owner"] = summary_df["repository"].apply(lambda x: x.split("/")[-2])
summary_df["repo_name"] = summary_df["repository"].apply(lambda x: x.split("/")[-1])



In [82]:

# Drop original column
summary_df = summary_df.drop(columns=["intervals"])

In [83]:
# desired front columns
front_cols = ["owner", "repo_name"]

# all other columns except these
other_cols = [c for c in summary_df.columns if c not in front_cols + ["repository"]]

# reorder
summary_df = summary_df[front_cols + other_cols]

In [84]:
summary_df.to_csv("fixed_workflows_vuln_durations.csv", index=False)